# 1. 데이터 로드

In [1]:
import os
import sys
import time
import random
import datetime
import requests
import pandas as pd
import numpy as np
import hashlib, hmac, base64
from itertools import combinations, permutations
from dtw import *
import json
import urllib.request
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
import pickle
from pytz import timezone
from difflib import SequenceMatcher

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from collections import defaultdict
from pytrends.request import TrendReq


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



# 2. API설정

In [2]:
from api_set import APIClient

# API 설정
from utils import get_secret
BASE_URL = get_secret("BASE_URL")
CUSTOMER_ID = get_secret("CUSTOMER_ID")
API_KEY = get_secret("API_KEY")
SECRET_KEY = get_secret("SECRET_KEY")
URI = get_secret("URI")
METHOD = get_secret("METHOD")
# API 클라이언트 인스턴스 생성
api_client = APIClient(BASE_URL, CUSTOMER_ID, API_KEY, SECRET_KEY,URI,METHOD)


# 3. 연관검색어 수집

In [3]:
# 키 로드
from utils import load_keywords 
keywords_data = load_keywords('main_keyword.json')

from utils import get_today_date
# 오늘의 날짜 가져오기
formatted_today, day = get_today_date()


# 결과 저장 폴더 생성
from utils import make_directory

make_directory('./data')
make_directory('./data/rl_srch')
make_directory(f'./data/rl_srch/{day}')  # 키워드별 연관검색어 리스트 저장

In [4]:

# 검색어 리스트와 결과 저장 경로 설정
srch_keyword = ['keyword_final']  
save_path = './data/rl_srch/'  
print(api_client.base_url)

https://api.searchad.naver.com


In [5]:
srch_keyword

['keyword_final']

In [6]:
import os
import csv
import datetime
import asyncio
import pandas as pd

# 필요한 경우 비동기를 위한 nest_asyncio 적용
import nest_asyncio
nest_asyncio.apply()

from collect_keywords import collect_keywords

async def main(srch_keyword, day):
    # 오늘 날짜로 폴더 경로 생성
    folder_path = './data/rl_srch/' + datetime.datetime.now().strftime('%y%m%d')
    file_path = f"{folder_path}/collected_keywords.csv"
    
    # 폴더가 존재하는지 확인
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 파일이 존재하는지 확인
    if os.path.isfile(file_path):
        # 파일이 존재하면, 데이터를 읽어옵니다.
        collected_keywords_data = pd.read_csv(file_path)
    else:
        # 파일이 없으면, collect_keywords 함수를 호출해서 데이터를 수집합니다.
        collected_keywords_data = await collect_keywords(srch_keyword, day)
        # 결과를 CSV로 저장
        collected_keywords_data.to_csv(file_path, index=False)
    
    return collected_keywords_data
collected_keywords_data=asyncio.run(main(srch_keyword, day))

In [7]:
print(collected_keywords_data)

         연관키워드  월간검색수_합계 검색어
0           주식  528500.0  주식
1          미주부     370.0  주식
2        김현준대표     940.0  주식
3         퀀트투자    6650.0  주식
4         주식투자   12310.0  주식
...        ...       ...  ..
30599   수원유언공증      18.0  증여
30600   법인주소변경    1940.0  증여
30601   인터넷법무사     460.0  증여
30602    환지예정지     330.0  증여
30603  지정유언집행자      80.0  증여

[30604 rows x 3 columns]


In [8]:
sorted_df = collected_keywords_data.groupby('검색어').apply(lambda x: x.sort_values('월간검색수_합계', ascending=False)).reset_index(drop=True)

# 각 '검색어'별로 분리된 DataFrame을 리스트에 저장
grouped = sorted_df.groupby('검색어')
df_list = [group for _, group in grouped]
print(len(df_list))

45


C:\Users\chohy\AppData\Local\Temp\ipykernel_37188\3376418344.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_df = collected_keywords_data.groupby('검색어').apply(lambda x: x.sort_values('월간검색수_합계', ascending=False)).reset_index(drop=True)


In [9]:
from utils import merge_and_mark_duplicates_limited
collected_keywords_data = merge_and_mark_duplicates_limited(df_list)


collected_keywords_data



,연관키워드,월간검색수_합계,검색어,중복검색어
0,달러환율,2050600.0,CMA금리비교,"CMA금리비교,WTI,달러환율,미국금리,미국주식,주가지수,주식"
1,적금계산기,644900.0,CMA금리비교,"CMA금리비교,CMA통장"
2,다우지수,561700.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,급등주,달러환율,미국금리,미국주식,테마주"
3,적금,470400.0,CMA금리비교,"CMA금리비교,CMA통장,금리,급등주,돈버는법,미국금리,배당주,재테크,주가지수,주식"
4,코스닥,450600.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,개인연금,달러환율,배당주,신규상장,외국인순매..."
...,...,...,...,...
1141,S&P500지수,33530.0,테마주,"테마주,퇴직연금"
1142,TQQQ,32650.0,테마주,테마주
1143,코스피야간선물,31270.0,테마주,테마주
1144,양자컴퓨터관련주,30430.0,테마주,테마주


In [26]:
import utils

def add_client_info(collected_keywords_data, start_id_index=1):
    clients = utils.get_secret("clients")
    start_id_index = 1
    clients = utils.get_secret("clients")
    # ID와 PW 컬럼을 데이터프레임에 추가하는 로직
    total_rows = len(collected_keywords_data)
    ids = []
    pws = []

    for i in range(total_rows):
        # 현재 id 인덱스 계산 (start_id_index를 기준으로)
        current_id_index = ((i // 500) + start_id_index) % len(clients)
        current_id_key = f"id_{current_id_index}"
        
        # 현재 id와 pw 할당
        current_id = clients[current_id_key]['client_id']
        current_pw = clients[current_id_key]['client_secret']
        
        ids.append(current_id)
        pws.append(current_pw)

    # ID와 PW 컬럼 추가
    collected_keywords_data['id'] = ids
    collected_keywords_data['pw'] = pws

    return collected_keywords_data
collected_keywords_data= add_client_info(collected_keywords_data)
collected_keywords_data

,연관키워드,월간검색수_합계,검색어,중복검색어,id,pw
0,달러환율,2050600.0,CMA금리비교,"CMA금리비교,WTI,달러환율,미국금리,미국주식,주가지수,주식",eywsrQuPMFtlbYz1XAl7,Me4ra4s8od
1,적금계산기,644900.0,CMA금리비교,"CMA금리비교,CMA통장",eywsrQuPMFtlbYz1XAl7,Me4ra4s8od
2,다우지수,561700.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,급등주,달러환율,미국금리,미국주식,테마주",eywsrQuPMFtlbYz1XAl7,Me4ra4s8od
3,적금,470400.0,CMA금리비교,"CMA금리비교,CMA통장,금리,급등주,돈버는법,미국금리,배당주,재테크,주가지수,주식",eywsrQuPMFtlbYz1XAl7,Me4ra4s8od
4,코스닥,450600.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,개인연금,달러환율,배당주,신규상장,외국인순매...",eywsrQuPMFtlbYz1XAl7,Me4ra4s8od
...,...,...,...,...,...,...
1141,S&P500지수,33530.0,테마주,"테마주,퇴직연금",mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
1142,TQQQ,32650.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
1143,코스피야간선물,31270.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
1144,양자컴퓨터관련주,30430.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8


In [27]:
def groupped_df(name,collected_keywords_data):
    grouped = collected_keywords_data.groupby(name)
    df_list = [group for _, group in grouped]
    return df_list
df_list=groupped_df('id',collected_keywords_data)
print(len(df_list))

3


In [28]:
df_list[1]

,연관키워드,월간검색수_합계,검색어,중복검색어,id,pw
1000,근로기준법,50500.0,적립식펀드,"적립식펀드,퇴직연금",mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
1001,근로계약서,40400.0,적립식펀드,"적립식펀드,퇴직연금",mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
1002,취득세,40100.0,적립식펀드,"적립식펀드,퇴직연금",mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
1003,미국달러환율,35110.0,적립식펀드,적립식펀드,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
1004,배당주순위,21920.0,적립식펀드,적립식펀드,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
...,...,...,...,...,...,...
1141,S&P500지수,33530.0,테마주,"테마주,퇴직연금",mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
1142,TQQQ,32650.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
1143,코스피야간선물,31270.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8
1144,양자컴퓨터관련주,30430.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8


# 4. 키워드 트렌드 및 패턴 분석

1) 정보지정

In [12]:
import utils
from datetime import datetime
    # 날짜 지정
today = datetime.now(timezone('Asia/Seoul'))
day = today.strftime("%y%m%d")  

    # 저장할 폴더 미리 생성, 폴더가 이미 있으면 생성하지 않음
folders_to_create = [
        './data/tmp',
        './data/result',
        f'./data/result/{day}',
        f'./data/result/{day}/graph'
    ]

for folder in folders_to_create:
        os.makedirs(folder, exist_ok=True)

    # 분석 기간 설정
start_index = (today - relativedelta(days=1)).strftime("%Y-%m-%d")
end_index = (today - relativedelta(years=3) - relativedelta(days=1)).strftime("%Y-%m-%d")
print(['분석시작일:', start_index, '분석종료일:', end_index])

    # URI 정보 가져오기
URI = utils.get_secret("URI")

['분석시작일:', '2024-03-06', '분석종료일:', '2021-03-06']


# 5. 상세 분석

In [14]:
# 삭제할 파일 경로 정의
progress_log_path = 'progress_log.txt'  # 가정: 실제 경로에 맞게 수정해야 함
progress_state_path = 'progress_state.json'  # 가정: 실제 경로에 맞게 수정해야 함

# progress_log 파일 삭제
if os.path.exists(progress_log_path):
    os.remove(progress_log_path)
    print(f'{progress_log_path} has been deleted.')
else:
    print(f'{progress_log_path} does not exist.')

# progress_state.json 파일 삭제
if os.path.exists(progress_state_path):
    os.remove(progress_state_path)
    print(f'{progress_state_path} has been deleted.')
else:
    print(f'{progress_state_path} does not exist.')

progress_log.txt has been deleted.
progress_state.json does not exist.


trend_data : 상대지표


In [11]:
import asyncio
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import utils
from trend import trend_maincode
import time
from pytz import timezone
import select_keyword

class analysis:
    def __init__(self):
        # 기본설정
        self.formatted_today, self.day=utils.get_today_date()
        self.state=utils.load_state()
        self.apiCallCount = self.state.get('api_request_count', 1)
        self.current_client_index = self.state.get('current_client_index', 0)
        self.keywords = utils.load_keywords('main_keyword.json')['keyword_final']
        self.request_limit = 1000
        self.keyword_index = self.state['keyword_index']
        self.standard_time = datetime.now()
        self.api_url = "https://openapi.naver.com/v1/datalab/search"
        self.today = datetime.now(timezone('Asia/Seoul'))
        self.api_request_data = pd.DataFrame(columns=['search_keywords'])
        self.start_index = (self.today - relativedelta(days=1)).strftime("%Y-%m-%d")
        self.end_index = (self.today - relativedelta(years=3) - relativedelta(days=1)).strftime("%Y-%m-%d")
        self.clients = utils.get_secret("clients")
        

        self.api_request_data_dataname = {
                'api_request_data': [],
                'keyname': []
        }

        # 데이터프레임 초기화
        self.trends_dataframes = {
        'daily_up': pd.DataFrame(),
        'weekly_up': pd.DataFrame(),
        'weekly_stay': pd.DataFrame(),
        'monthly_up': pd.DataFrame(),
        'monthly_stay': pd.DataFrame(),
        'monthly_rule': pd.DataFrame()
        }


        self.graph_tables = {
        'day': pd.DataFrame(index=pd.date_range(start=self.start_index, end=self.end_index, freq='1d')),
        'week': pd.DataFrame(index=pd.date_range(start=self.start_index, end=self.end_index, freq='7d')),
        'month': pd.DataFrame(index=pd.date_range(start=self.start_index, end=self.end_index, freq='28d'))
        }

        # 정보 저장을 위한 딕셔너리 초기화
        self.keyword_data = {
            'keyword_categories': {},
            'related_search_terms': {},
            'related_search_volume': {},
            'up_month': {}
        }
        self.review_types = {
                'daily': {
                        'function': select_keyword.select_keyword,
                        'info_dict': {},
                        'trend_df': self.trends_dataframes['daily_up'],
                        'graph_df': self.graph_tables['day'],
                        'time_period': 'daily'
                    },
            'weekly_up': {
                        'function': select_keyword.select_keyword,
                        'info_dict': {},
                        'trend_df': self.trends_dataframes['weekly_up'],
                        'graph_df': self.graph_tables['week'],
                        'time_period': 'weekly'
                    },
            'weekly_stay': {
                        'function': select_keyword.rising_keyword_analysis,
                        'info_dict': {},
                        'trend_df': self.trends_dataframes['weekly_stay'],
                        'graph_df': self.graph_tables['month'],
                        'time_period': 'weekly'
                    },
            'monthly_up': {
                        'function': select_keyword.select_keyword,
                        'info_dict': {},
                        'trend_df': self.trends_dataframes['monthly_up'],
                        'graph_df': self.graph_tables['month'],
                        'time_period': 'month'
                    },
            'monthly_stay': {
                        'function': select_keyword.rising_keyword_analysis,
                        'info_dict': {},
                        'trend_df': self.trends_dataframes['monthly_stay'],
                        'graph_df': self.graph_tables['month'],
                        'time_period': 'month'
                    },
            'monthly_rule': {
                        'function': select_keyword.monthly_rule,
                        'info_dict': {},
                        'trend_df': self.trends_dataframes['monthly_rule'],
                        'graph_df': self.graph_tables['month'],
                        'time_period': 'month'
                    }
                }
        pass


        # api 설정
    def handle_api_call(self,keywordName,df_table,currentRequestCount):
        if self.apiCallCount<= self.request_limit:
            id_num,pw,_=utils.get_client_info(self.clients,self.current_client_index)
            id = self.clients[id_num]['client_id']
            pw=self.clients[id_num]['client_secret']
            utils.log_progress(keywordName, currentRequestCount, \
                               len(df_table), id, self.apiCallCount, self.request_limit)
            related_keyword = df_table['연관키워드'][currentRequestCount]
            request_data = {
            
            'search_keywords': related_keyword,

            }        
            return request_data, self.apiCallCount + 1, self.current_client_index
        else:
                # 요청 한도 초과 시 클라이언트 인덱스 업데이트
                self.current_client_index += 1
                if self.current_client_index >= len(self.clients):
                    print("모든 API 클라이언트의 요청 한도 초과")
                    return None, self.apiCallCount, self.current_client_index  # 처리 중단을 위한 None 반환
                else:
                    # 클라이언트 전환 후 재시도
                    return self.handle_api_call(keywordName, currentRequestCount, df_table)
                
    # 연관 검색어의 trend 데이터
    def analyze_trend_data(self, uniq):
        params = {
            "search_keywords": uniq["api_request_data"],  
            "id": self.clients['id_1']["client_id"],
            "pw": self.clients['id_1']["client_secret"],
            "api_url": self.api_url
        }
        trend_data = asyncio.run(trend_maincode(params,self.clients, self.api_url))
        return trend_data
    
    # 연관검색어의 급상승, 지속상승 로직 적용
    def select_keywords(self, trend_data):
        review_settings = {key: [] for key in self.review_types.keys()}
        for i, (index, row) in enumerate(trend_data.iterrows()):
            # 각 행의 첫 번째 열과 두 번째 열의 값 참조
            first_column_value = row['trend_data']
            second_column_value = row['keyname']
            
            for review_key, settings in self.review_types.items():
                if review_key == 'monthly_rule':
                    # monthly_rule에 대한 특별 처리
                    trend_analysis_df, graph_data_df, analysis_info, rising_month = settings['function'](first_column_value, self.day, settings['time_period'])
                    if trend_analysis_df is not None:
                        review_settings[review_key].append((trend_analysis_df, graph_data_df, analysis_info, rising_month, second_column_value,first_column_value.columns[0]))
                else:
                    # 기타 경우 처리
                    trend_analysis_df, graph_data_df, analysis_info = settings['function'](first_column_value, self.day, settings['time_period'])
                    if trend_analysis_df is not None:
                        review_settings[review_key].append((trend_analysis_df, graph_data_df, analysis_info, second_column_value,first_column_value.columns[0]))
        return review_settings
                            # tmp 랑 tmp_gph, tmp_info 얘네를 딕셔너리에 집어넣는 함수

    #     #딕셔너리에 데이터를 넣는 함수
    def insert_data_into_dict(self):
        
        pass



    def collect_and_analyze_keyword_trends(self,df_list):
        print(f'키워드 수: {len(df_list)}')
        total_keywords = len(df_list)
        api_request_data_list = []  # DataFrame 대신 사용할 리스트
        for df_table in df_list:
            for keywordIndex, keywordName in enumerate(self.keywords[self.keyword_index:], start=self.keyword_index):
                print(f'################################################ {keywordName} ({keywordIndex+1}/{total_keywords}) ################################################')
                maxKeyword = min(50, len(df_table))

                for currentRequestCount in range(self.state['currentRequestCount_index'], maxKeyword):
                    request_data, self.apiCallCount, self.current_client_index = self.handle_api_call(keywordName, df_table, currentRequestCount)
                    self.api_request_data_dataname['keyname'].append(keywordName)
                    # API 요청 예외 처리 (한도 초과 시 클라이언트 인덱스 업데이트)
                    if request_data is not None:
                        api_request_data_list.append(request_data)  # 리스트에 데이터 추가
                    else:
                        break

                if self.current_client_index >= len(self.clients):
                    print("모든 API 클라이언트의 요청 한도 초과")
                    break

            self.keyword_index += 1

        # 리스트를 DataFrame으로 변환
            
        if api_request_data_list:
            self.api_request_data = pd.DataFrame(api_request_data_list)
            api_data_list = self.api_request_data['search_keywords'].tolist()

            self.api_request_data_dataname['api_request_data'] = api_data_list
        print(f'API 요청 데이터 수: {len(self.api_request_data)}')

        # 실시간 급상승 이런거 들어가는 함수부분
  

In [12]:


start=time.time()
analysis_instance = analysis()
trend_data=analysis_instance.collect_and_analyze_keyword_trends(df_list)
print(time.time()-start)

키워드 수: 43
################################################ 주식 (1/43) ################################################
################################################ 금리 (2/43) ################################################
Current Client ID: eywsrQuPMFtlbYz1XAl7 (100/1000) , index 49 of 50

################################################ 금융상품 (3/43) ################################################
################################################ 디지털자산 (4/43) ################################################
Current Client ID: eywsrQuPMFtlbYz1XAl7 (200/1000) , index 49 of 50

################################################ 부동산 (5/43) ################################################
################################################ 세금 (6/43) ################################################
Current Client ID: eywsrQuPMFtlbYz1XAl7 (300/1000) , index 49 of 50

################################################ 재테크 (7/43) ################################################
####################

AttributeError: module 'utils' has no attribute 'merge_keys_for_unique_names'

In [11]:
result_dict=analysis_instance.select_keywords(trend_data)


월별 급상승 키워드 발견 : ISA계좌
월별 급상승 키워드 발견 : AI관련주
주별 지속상승 키워드 발견 : ETF
일별 급상승 키워드 발견: 퇴직연금
월별 급상승 키워드 발견 : IRP계좌개설
월별 지속상승 키워드 발견 : IRP계좌개설
월별 급상승 키워드 발견 : CMA통장
월별 지속상승 키워드 발견 : CMA통장
일별 급상승 키워드 발견: ETF투자방법
월별 급상승 키워드 발견: 배당주
월별 급상승 키워드 발견 : 파킹통장
일별 급상승 키워드 발견: 비트코인전망
주별 급상승 키워드 발견: 비트코인전망
월별 급상승 키워드 발견 : 파킹통장금리비교
월별 지속상승 키워드 발견 : 파킹통장금리비교
일별 급상승 키워드 발견: 샌드박스코인
주별 급상승 키워드 발견: 샌드박스코인
월별 급상승 키워드 발견 : 해외주식


In [61]:
result_dict['monthly_rule']

[]

In [70]:
mode_list=['daily','weekly_up','weekly_stay','monthly_up','monthly_stay','monthly_rule']
def assign_value_to_result_dict(mode, num, value):
    """result_dict의 각 항목 끝에 value를 추가합니다."""
    # 현재 항목을 리스트로 변환 (튜플일 경우)
    current_item = list(result_dict[mode][num])
    # value를 리스트 끝에 추가
    current_item.append(value)
    # 변경된 리스트를 다시 result_dict에 할당
    result_dict[mode][num] = current_item


for mode in mode_list:
    n = result_dict[mode]
    for num in range(len(n)):
        keyword = result_dict[mode][num][4] if mode != 'monthly_rule' else result_dict[mode][num][5]
        keyword_exists = (collected_keywords_data['연관키워드'] == keyword).any()
        
        if keyword_exists:
            filtered_data = collected_keywords_data[collected_keywords_data['연관키워드'] == keyword]
            if not filtered_data.empty:  # 필터링된 데이터프레임이 비어있지 않은지 확인
                value = filtered_data['월간검색수_합계'].iloc[0]  # 원하는 값
                assign_value_to_result_dict(mode, num, value)  # 값을 result_dict에 할당



result_dict['daily']

[[            기준일자     유형 연관검색어       검색일자        검색량
  0     2024-03-06  일별급상승  퇴직연금 2021-03-05   37.96186
  1     2024-03-06  일별급상승  퇴직연금 2021-03-06   17.85051
  2     2024-03-06  일별급상승  퇴직연금 2021-03-07   19.38586
  3     2024-03-06  일별급상승  퇴직연금 2021-03-08   44.54192
  4     2024-03-06  일별급상승  퇴직연금 2021-03-09   44.87936
  ...          ...    ...   ...        ...        ...
  1092  2024-03-06  일별급상승  퇴직연금 2024-03-01   20.78623
  1093  2024-03-06  일별급상승  퇴직연금 2024-03-02   16.34891
  1094  2024-03-06  일별급상승  퇴직연금 2024-03-03   16.85507
  1095  2024-03-06  일별급상승  퇴직연금 2024-03-04   76.93605
  1096  2024-03-06  일별급상승  퇴직연금 2024-03-05  100.00000
  
  [1097 rows x 5 columns],
              기준일자     유형 연관검색어       검색일자        검색량
  0     2024-03-06  일별급상승  퇴직연금 2021-03-05   37.96186
  1     2024-03-06  일별급상승  퇴직연금 2021-03-06   17.85051
  2     2024-03-06  일별급상승  퇴직연금 2021-03-07   19.38586
  3     2024-03-06  일별급상승  퇴직연금 2021-03-08   44.54192
  4     2024-03-06  일별급상승  퇴직연금 2021-03-09   44.8793

In [ ]:
keyword = result_dict[mode][num][4] if mode != 'monthly_rule' else result_dict[mode][num][5]

In [ ]:
mode_list=['daily','weekly_up','weekly_stay','monthly_up','monthly_stay','monthly_rule']
def assign_value_to_result_dict(mode, num, value):
    """result_dict의 각 항목 끝에 value를 추가합니다."""
    # 현재 항목을 리스트로 변환 (튜플일 경우)
    current_item = list(result_dict[mode][num])
    # value를 리스트 끝에 추가
    current_item.append(value)
    # 변경된 리스트를 다시 result_dict에 할당
    result_dict[mode][num] = current_item


for mode in mode_list:
    n = result_dict[mode]
    for num in range(len(n)):
        keyword = ...  # keyword 결정 로직 (예시 데이터 필요)
        keyword_exists = (collected_keywords_data['연관키워드'] == keyword).any()
        
        if keyword_exists:
            filtered_data = collected_keywords_data[collected_keywords_data['연관키워드'] == keyword]
            if not filtered_data.empty:  # 필터링된 데이터프레임이 비어있지 않은지 확인
                value = filtered_data['월간검색수_합계'].iloc[0]  # 원하는 값
                print(1)
                assign_value_to_result_dict(mode, num, value)  # 값을 result_dict에 할당



result_dict['daily']

In [71]:
result_dict['daily']

[[            기준일자     유형 연관검색어       검색일자        검색량
  0     2024-03-06  일별급상승  퇴직연금 2021-03-05   37.96186
  1     2024-03-06  일별급상승  퇴직연금 2021-03-06   17.85051
  2     2024-03-06  일별급상승  퇴직연금 2021-03-07   19.38586
  3     2024-03-06  일별급상승  퇴직연금 2021-03-08   44.54192
  4     2024-03-06  일별급상승  퇴직연금 2021-03-09   44.87936
  ...          ...    ...   ...        ...        ...
  1092  2024-03-06  일별급상승  퇴직연금 2024-03-01   20.78623
  1093  2024-03-06  일별급상승  퇴직연금 2024-03-02   16.34891
  1094  2024-03-06  일별급상승  퇴직연금 2024-03-03   16.85507
  1095  2024-03-06  일별급상승  퇴직연금 2024-03-04   76.93605
  1096  2024-03-06  일별급상승  퇴직연금 2024-03-05  100.00000
  
  [1097 rows x 5 columns],
              기준일자     유형 연관검색어       검색일자        검색량
  0     2024-03-06  일별급상승  퇴직연금 2021-03-05   37.96186
  1     2024-03-06  일별급상승  퇴직연금 2021-03-06   17.85051
  2     2024-03-06  일별급상승  퇴직연금 2021-03-07   19.38586
  3     2024-03-06  일별급상승  퇴직연금 2021-03-08   44.54192
  4     2024-03-06  일별급상승  퇴직연금 2021-03-09   44.8793

In [15]:
print(type(a))

<class 'dict'>


In [13]:
review = analysis_instance.keyword_data()

{'keyword_categories': {},
 'related_search_terms': {},
 'related_search_volume': {},
 'up_month': {}}

### 동기 : 49.392563343048096


In [10]:
data_list=['daily','weekly_up','weekly_stay','monthly_up','monthly_stay','monthly_rule']
daily_up_trend, weekly_up_trend, weekly_stay_trend, monthly_up_trend, monthly_stay_trend, monthly_rule_trend = [pd.DataFrame() for _ in range(6)]

In [11]:
daily_up_info, weekly_up_info, weekly_stay_info, monthly_up_info, monthly_stay_info, monthly_rule_info = [{} for _ in range(6)]

In [12]:
daily_up_trendc=analysis.map_columns_based_on_keys(review_types, 'daily')

weekly_up_trend=analysis.map_columns_based_on_keys(review_types, 'weekly_up')

weekly_stay_trend=analysis.map_columns_based_on_keys(review_types, 'weekly_stay')

monthly_up_trend=analysis.map_columns_based_on_keys(review_types, 'monthly_up')

monthly_stay_trend=analysis.map_columns_based_on_keys(review_types, 'monthly_stay')

monthly_rule_trend=analysis.map_columns_based_on_keys(review_types, 'monthly_rule')
    

One or more required columns ['검색일자', '연관검색어', '검색량'] are missing in the DataFrame.
One or more required columns ['검색일자', '연관검색어', '검색량'] are missing in the DataFrame.
One or more required columns ['검색일자', '연관검색어', '검색량'] are missing in the DataFrame.
One or more required columns ['검색일자', '연관검색어', '검색량'] are missing in the DataFrame.


In [13]:
dict_list = ['keyword_categories', 'related_search_terms', \
             'related_search_volume', 'daily_up_info',\
                  'weekly_up_info', 'weekly_stay_info', 'monthly_up_info', \
                    'monthly_stay_info', 'monthly_rule_info', 'up_month']

trend_type = dict(zip(['daily_up_trend', 'weekly_up_trend', \
                       'weekly_stay_trend', 'monthly_up_trend', 'monthly_stay_trend', 'monthly_rule_trend'] , 
                        ['일별 급상승', '주별 급상승', '주별 지속상승', '월별 급상승', '월별 지속상승', '월별 규칙성']))



In [14]:
# 결과 저장을 위한 폴더 경로
folders = ['./data/dict', './data/types', './data/json']

try:
    # 폴더 생성
    for folder in folders:
        if not os.path.exists(folder):
            os.makedirs(folder)
        else:
            print(f"Folder '{folder}' already exists.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

Folder './data/dict' already exists.
Folder './data/types' already exists.
Folder './data/json' already exists.


In [15]:
import pickle

# 유형 분류 결과 저장
for k in dict_list:
    with open(f'./data/dict/{k}.pkl', 'wb') as f:
        pickle.dump(eval(f'{k}'), f)

for type_eng, type_kor in trend_type.items():
    evaluated_df = eval(type_eng)  # 변수의 평가된 결과를 저장
    if evaluated_df is not None:  # 평가된 결과가 None이 아니면 CSV로 저장
        print(f"{type_eng}에 대한 데이터프레임이 존재합니다. CSV 파일로 저장합니다.")
        evaluated_df.to_csv(f'./data/types/{type_eng}_result.csv', encoding='utf-8-sig')
    else:  # 평가된 결과가 None이면 오류 메시지 출력
        print(f"Error for type_eng '{type_eng}': 'NoneType' object has no attribute 'to_csv'")


daily_up_trend에 대한 데이터프레임이 존재합니다. CSV 파일로 저장합니다.
weekly_up_trend에 대한 데이터프레임이 존재합니다. CSV 파일로 저장합니다.
Error for type_eng 'weekly_stay_trend': 'NoneType' object has no attribute 'to_csv'
monthly_up_trend에 대한 데이터프레임이 존재합니다. CSV 파일로 저장합니다.
Error for type_eng 'monthly_stay_trend': 'NoneType' object has no attribute 'to_csv'
Error for type_eng 'monthly_rule_trend': 'NoneType' object has no attribute 'to_csv'


# 파일 정리 코드


make_directory('./data')
make_directory('./data/rl_srch')
make_directory(f'./data/rl_srch/{day}')  # 키워드별 연관검색어 리스트 저장

이걸로 생성된 파일 삭제하는 코드

In [3]:
from utils import remove_directory

# 특정 디렉토리 삭제    
remove_directory('./data')

디렉토리 './data'가 삭제되었습니다.


# 로그파일 삭제

In [13]:
import multiprocessing

# 사용 가능한 CPU 코어의 수를 확인
cpu_count = multiprocessing.cpu_count()
print(f"Available CPU cores: {cpu_count}")

Available CPU cores: 16


1. 세션이 끊겼을 때 어떻게해야 좋을지 판단 로그파일을 인터넷에 깔아서 관리할까? 아니면 딕셔너리로 보이게 해서 관리할까? 라는 생각
2. 너무 하드코딩되어있는 부분 존재 이런부분은 처리하기
3. missing데이터 같은 경우 딕셔너리로 관리해서 daily나 weekly일때 다르게 구분하기 편하게 하기
4. 로그파일로 따로 관리하는거 좋은가? 에 대한 생각해보기
5. 주석같은거 중요한거마다 하기
6. 